In [17]:
import numpy as np
import cv2 
import os
from glob import glob
import matplotlib.pyplot as plt

from keras.applications import ResNet50
from keras.models import Model
import copy
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

2.10.0


In [2]:
images_path = 'Flickr_Data/Images/'
images = glob(images_path+'*.jpg')
captions_path = "Flickr_Data/Flickr_TextData/FLickr8k.token.txt"

In [3]:
resnet_model = ResNet50(include_top=True)

In [4]:
# restructure model
resnet_model = Model(inputs=resnet_model.input, outputs=resnet_model.layers[-2].output)

In [ ]:
# preprocess the images
img_feature_vectors = {}

count = 0
for item in images:
    img = cv2.imread(item)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img.reshape(1, 224, 224, 3)
    
    feature_vector = resnet_model.predict(img, verbose=0).reshape(2048,)
    
    img_filename = item.split('\\')[-1]
    img_feature_vectors[img_filename] = feature_vector
    
    count += 1
    
    if (count%50==0):
        print(count)
    
    if (count==1499):
        break

In [ ]:
np.save('image_feature_vectors.npy', img_feature_vectors)

In [5]:
# load in dictionary of image feature vectors
img_feature_vectors = np.load('image_feature_vectors.npy', allow_pickle='TRUE').item()

In [6]:
# makes a string lowercase, prepends it with the string 'sos' and appends with 'eos'
def process_string(s):
    s = s.lower()
    return 'sos ' + s + ' eos'

In [7]:
# preprocess the captions data
captions_dict = {}

f = open(captions_path, 'r').read().split('\n')

# generate a dictionary of filenames to a list of captions
for line in f:
    try:
        filename_caption = line.split('\t') 
        filename = filename_caption[0][:-2]
        caption = process_string(filename_caption[1])
        
        if filename in img_feature_vectors:
            if filename not in captions_dict:
                captions_dict[filename] = [caption]
            else:
                captions_dict[filename].append(caption)
    except:
        pass

In [8]:
# generate a dictionary called vocab which will contain all words in the set of captions mapped to a unique integer
vocab = {}

count = 1
for filename, captions in captions_dict.items():
    for caption in captions:
        caption_as_list = caption.split()
        for word in caption_as_list:
            if word not in vocab:
                vocab[word] = count
                count += 1

In [9]:
# takes a string and returns a list of integers where each integer corresponds to a particular word
def encode_string(s, adict):
    s_list = s.split()
    encoded_string = []
    for word in s_list:
        encoded_string.append(adict[word])
    return encoded_string

In [10]:
# generates a copy of captions_dict where each caption is replaced with a list of integers where each integer corresponds to a word in the caption
captions_dict_encoded = copy.deepcopy(captions_dict)

for filename, captions in captions_dict_encoded.items():
    for i, caption in enumerate(captions):
        captions[i] = encode_string(caption, vocab)

In [11]:
# get length of the longest caption in the data set
max_len = 0
for filename, captions in captions_dict_encoded.items():
    for caption in captions:
        if len(caption)>max_len:
            max_len = len(caption)

In [12]:
captions_dict_encoded = np.load('captions_dict_encoded.npy', allow_pickle='TRUE').item()

In [13]:
def generate_training_data(image_feature_vectors, captions_dict_encoded):
    X = []
    y_in = []
    y_out = []
    
    for filename, captions in captions_dict_encoded.items():
        for caption in captions:
            i = 0
            for word in caption:
                y_in_item = caption[:i]
                y_in_item = (y_in_item + max_len * [0])[:max_len]
                y_in.append(y_in_item)
                
                y_out_item = [0]*len(vocab)
                y_out_item[word-1] = 1
                y_out.append(y_out_item)
                
                X_item = image_feature_vectors[filename]
                X.append(X_item)
                i+=1
    return X, y_in, y_out

In [14]:
import time
start_time = time.time()
X, y_in, y_out = generate_training_data(img_feature_vectors, captions_dict_encoded)
print(time.time()-start_time)

4.333067417144775


In [16]:
X = np.array(X)
y_in = np.array(y_in, dtype='float64')
y_out = np.array(y_out, dtype="float64")

NameError: name 'X' is not defined

In [47]:
np.save('X.npy', X)
np.save('y_in.npy', y_in)
np.save('y_out.npy', y_out)

In [15]:
del X
del y_in
del y_out

In [63]:
X = np.load('X.npy', allow_pickle='TRUE')
y_in = np.load('y_in.npy', allow_pickle='TRUE')
y_out = np.load('y_out.npy', allow_pickle='TRUE')

In [20]:
embedding_size = 128
vocab_len = len(vocab)

#image
x = keras.Sequential([
    keras.layers.Input(shape=(2048,)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.RepeatVector(max_len)
])

#caption
y = keras.Sequential([
    keras.layers.Embedding(vocab_len, embedding_size, input_length=max_len),
    keras.layers.LSTM(embedding_size, return_sequences=True)
])

#concatinate outputs of y and x
z = keras.layers.Concatenate()([x.output, y.output])
z = keras.layers.LSTM(128, return_sequences=True)(z)
z = keras.layers.LSTM(512, return_sequences=False)(z)
z = keras.layers.Dense(vocab_len, activation='softmax')(z)

model = Model(inputs=[x.input, y.input], outputs=z)